In [2]:
import tensorflow as tf
import keras
from keras import layers
# from utils.data_preprocessing import load_data_artifact

In [3]:
def load_data_artifact(dir_path):
    train, valid = tf.keras.preprocessing.image_dataset_from_directory(
        dir_path,
        label_mode='binary',
        image_size=[200, 200],
        validation_split=0.2,
        subset='both',
        shuffle=True,
        seed=42,
        interpolation='area',
        batch_size=256
    )

    return train, valid

In [4]:
def create_model():
    backbone = keras.applications.VGG19(False, 'imagenet', input_shape=(200, 200, 3))

    for layer in backbone.layers:
        layer.trainable = False

    flatten = layers.Flatten()(backbone.output)
    dense = layers.Dense(256, activation='elu')(flatten)
    norm = layers.BatchNormalization()(dense)

    dense = layers.Dense(256, activation='elu')(norm)
    norm = layers.BatchNormalization()(dense)

    dense = layers.Dense(256, activation='elu')(norm)
    norm = layers.BatchNormalization()(dense)

    dense = layers.Dense(1, activation='sigmoid')(norm)

    adam = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model = tf.keras.Model(inputs=backbone.input, outputs=dense)
    model.compile(
        optimizer=adam,
        loss='binary_crossentropy',
        metrics=['accuracy', 'mse', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    model.summary()
    return model

In [5]:
def train_model(model: tf.keras.Model, train_data, validation_data, model_name):
    # train_data = train_data.map(tf.keras.applications.vgg19.preprocess_input)
    # validation_data = validation_data.map(tf.keras.applications.vgg19.preprocess_input)

    saving_models = keras.callbacks.ModelCheckpoint("./saved_models/checkpoints/"+model_name+".{epoch:02d}.h5")
    tensorboard = keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)
    return model.fit(train_data, validation_data=validation_data, epochs=15, batch_size=256, callbacks=[saving_models, tensorboard])

In [53]:
model = create_model()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0   

In [13]:
train, valid = load_data_artifact("../../datasets/artifact")

Found 1934329 files belonging to 2 classes.
Using 1547464 files for training.
Using 386865 files for validation.


In [47]:
train2, valid2 = load_data_artifact("../../datasets/artifact")

Found 1934329 files belonging to 2 classes.
Using 1547464 files for training.
Using 386865 files for validation.


In [48]:
print(len(train2), len(valid2))

6045 1512


In [51]:
train_small = train2.take(40)
valid_small = valid2.take(40)

In [54]:
history = train_model(model, train_small, valid_small, "artifact-1")

Epoch 1/15
40/40 [==============================] - 437s 11s/step - loss: 0.7483 - accuracy: 0.5963 - mse: 0.2596 - precision_4: 0.5819 - recall_4: 0.5746 - val_loss: 0.8553 - val_accuracy: 0.5900 - val_mse: 0.2800 - val_precision_4: 0.5497 - val_recall_4: 0.8604
Epoch 2/15
40/40 [==============================] - 430s 11s/step - loss: 0.6060 - accuracy: 0.6829 - mse: 0.2060 - precision_4: 0.6762 - recall_4: 0.6599 - val_loss: 0.7390 - val_accuracy: 0.6290 - val_mse: 0.2482 - val_precision_4: 0.5828 - val_recall_4: 0.8294
Epoch 3/15
40/40 [==============================] - 424s 11s/step - loss: 0.5403 - accuracy: 0.7250 - mse: 0.1814 - precision_4: 0.7192 - recall_4: 0.7037 - val_loss: 0.6797 - val_accuracy: 0.6524 - val_mse: 0.2302 - val_precision_4: 0.6130 - val_recall_4: 0.7703
Epoch 4/15
40/40 [==============================] - 432s 11s/step - loss: 0.4873 - accuracy: 0.7651 - mse: 0.1606 - precision_4: 0.7594 - recall_4: 0.7500 - val_loss: 0.6564 - val_accuracy: 0.6644 - val_mse: 

KeyboardInterrupt: 